<a href="https://colab.research.google.com/github/Satwikram/Deep-Learning-Notebooks/blob/master/Transformers/Transformers%20for%20NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Author: Satwik Ram K

Implementaion of Transformer in NLP

French to English Translation

Imorting Dependencies

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import re
import math 
import time

In [2]:
from tensorflow.keras import layers
import tensorflow_datasets as tfds

In [4]:
tf.__version__

'2.2.0'

Loading data

In [5]:
with open("/content/drive/My Drive/Transformers/europarl-v7.fr-en.en", mode = 'r', encoding = 'utf-8') as f:
  euro_eng = f.read()

In [16]:
with open("/content/drive/My Drive/Transformers/europarl-v7.fr-en.fr", mode = 'r', encoding = 'utf-8') as f:
  euro_fr = f.read()

In [17]:
with open("/content/drive/My Drive/Transformers/P85-Non-Breaking-Prefix.en", mode = 'r', encoding = 'utf-8') as f:
  non_breaking_prefix_eng = f.read()

In [18]:
with open("/content/drive/My Drive/Transformers/P85-Non-Breaking-Prefix.fr", mode = 'r', encoding = 'utf-8') as f:
  non_breaking_prefix_fr = f.read()

In [19]:
euro_eng[:50]

'Resumption of the session\nI declare resumed the se'

Cleaning the data

In [20]:
non_breaking_prefix_eng = non_breaking_prefix_eng.split("\n")
non_breaking_prefix_eng = ['' + pref + '.' for pref in non_breaking_prefix_eng]

In [21]:
non_breaking_prefix_fr = non_breaking_prefix_fr.split("\n")
non_breaking_prefix_fr = ['' + pref + '.' for pref in non_breaking_prefix_fr]

In [22]:
non_breaking_prefix_eng

['a.',
 'b.',
 'c.',
 'd.',
 'e.',
 'f.',
 'g.',
 'h.',
 'i.',
 'j.',
 'k.',
 'l.',
 'm.',
 'n.',
 'o.',
 'p.',
 'q.',
 'r.',
 's.',
 't.',
 'u.',
 'v.',
 'w.',
 'x.',
 'y.',
 'z.',
 'messrs.',
 'mlle.',
 'mme.',
 'mr.',
 'mrs.',
 'ms.',
 'ph.',
 'prof.',
 'sr.',
 'st.',
 'a.m.',
 'p.m.',
 'vs.',
 'i.e.',
 'e.g.']

Tokenization

In [ ]:
corpus_eng = euro_eng
for prefix in non_breaking_prefix_eng:
  corpus_eng = corpus_eng.replace(prefix, prefix + '###')

corpus_eng = re.sub(r"\.(?=[A-Z]|[a-z]|[0-9])", ".###", corpus_eng)
corpus_eng = re.sub(r"\.###", "",  corpus_eng)
corpus_eng = re.sub(r"  +", "",  corpus_eng)
corpus_eng.split("\n")


In [ ]:
corpus_fr = euro_fr
for prefix in non_breaking_prefix_fr:
  corpus_fr = corpus_fr.replace(prefix, prefix + '###')

corpus_fr = re.sub(r"\.(?=[A-Z]|[a-z]|[0-9])", ".###", corpus_fr)
corpus_fr = re.sub(r"\.###", "",  corpus_fr)
corpus_fr = re.sub(r"  +", "",  corpus_fr)
corpus_fr.split("\n")

In [ ]:
tokenizer_eng = tfds.features.text.SubwordTextEncoder.build_from_corpus(
    corpus_eng, target_vocab_size = 2**13
)
tokenizer_fr = tfds.features.text.SubwordTextEncoder.build_from_corpus(
    corpus_fr, target_vocab_size = 2**13
)

In [ ]:
VOCAB_SIZE_ENG = tokenizer_en.vovab_size + 2
VOCAB_SIZE_FR = tokenizer_en.vovab_size + 2

In [ ]:
inputs = [[VOCAB_SIZE_EN-2] + tokenizer_en.encode(sentence) + [VOCAB_SIZE_EN-1]
          for sentence in corpus_eng]
outputs = [[VOCAB_SIZE_FR-2] + tokenizer_fr.encode(sentence) + [VOCAB_SIZE_FR-1]
           for sentence in corpus_fr]

Removing Longer Sentences

In [ ]:
MAX_LENGTH = 20
idx_to_remove = [count for count, sent in enumerate(inputs)
                 if len(sent) > MAX_LENGTH]
for idx in reversed(idx_to_remove):
    del inputs[idx]
    del outputs[idx]
idx_to_remove = [count for count, sent in enumerate(outputs)
                 if len(sent) > MAX_LENGTH]
for idx in reversed(idx_to_remove):
    del inputs[idx]
    del outputs[idx]

In [ ]:
inputs = tf.keras.preprocessing.sequence.pad_sequences(inputs,
                                                       value=0,
                                                       padding='post',
                                                       maxlen=MAX_LENGTH)
outputs = tf.keras.preprocessing.sequence.pad_sequences(outputs,
                                                        value=0,
                                                        padding='post',
                                                        maxlen=MAX_LENGTH)

In [ ]:
BATCH_SIZE = 64
BUFFER_SIZE = 20000

dataset = tf.data.Dataset.from_tensor_slices((inputs, outputs))

dataset = dataset.cache()
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)